In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import Control, Coupling, QuantumSimulator, QuantumSystem, Transmon

In [ ]:
transmons = [
    Transmon(
        label="Q08",
        dimension=3,
        frequency=7.646,
        anharmonicity=-0.356,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q09",
        dimension=3,
        frequency=8.442,
        anharmonicity=-0.448,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q10",
        dimension=3,
        frequency=8.316,
        anharmonicity=-0.412,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q11",
        dimension=3,
        frequency=7.192,
        anharmonicity=-0.368,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q08", "Q09"), strength=0.01),
    Coupling(pair=("Q10", "Q11"), strength=0.01),
    Coupling(pair=("Q08", "Q10"), strength=0.01),
    Coupling(pair=("Q09", "Q11"), strength=0.01),
]

system = QuantumSystem(
    objects=transmons,
    couplings=couplings,
)

In [ ]:
system.draw()

In [ ]:
system.hamiltonian

In [ ]:
simulator = QuantumSimulator(system)

In [ ]:
initial_state = system.state(
    {
        "Q08": "0",  # |0>
        "Q09": "-i",  # |-i>
        "Q10": "+",  # |+>
        "Q11": "*",  # random state in qubit subspace
    },
)

In [ ]:
duration = 30
tau = 10

pi_x = qp.FlatTop(
    duration=duration,
    amplitude=np.pi / (duration - tau),
    tau=tau,
)
pi_y = pi_x.shifted(np.pi / 2)

pulse = qp.PulseSequence([pi_x, pi_y])
pulse.plot()

In [ ]:
target = transmons[0]

detuning = 0.001

control = Control(
    target=target.label,
    frequency=target.frequency + detuning,
    waveform=pulse.values,
)
control.plot()

In [ ]:
result = simulator.mesolve(
    controls=[control],
    initial_state=initial_state,
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
final_state_target = result.get_substates("Q08")[-1]
final_state_target

In [ ]:
result.plot_population_dynamics("Q08")

In [ ]:
result.show_last_population("Q08")

In [ ]:
result.display_bloch_sphere("Q08")
result.display_bloch_sphere("Q09")
result.display_bloch_sphere("Q10")
result.display_bloch_sphere("Q11")